In [ ]:
# Colab: Make sure you select 'Runtime -> Change runtime type -> Hardware Accelerator: GPU' 

In [ ]:
# Install PGBM and ninja for CUDA extension compilation. 
%pip install pgbm
%pip install ninja

In [ ]:
#%% Load packages
import torch
from pgbm import PGBM
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt

In [ ]:
#%% Objective for pgbm
def mseloss_objective(yhat, y, levels=None):
    gradient = (yhat - y)
    hessian = torch.ones_like(yhat)

    return gradient, hessian

def rmseloss_metric(yhat, y, levels=None):
    loss = (yhat - y).pow(2).mean().sqrt()

    return loss

In [ ]:
#%% Load data
X, y = load_boston(return_X_y=True)
params = {'device':'gpu'}

In [ ]:
#%% Train pgbm
# Split data
torch_device = torch.device('cpu') if params['device'] == 'cpu' else torch.device(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
torch_data = lambda x: torch.from_numpy(x).float().to(torch_device)
train_data = (torch_data(X_train), torch_data(y_train))
test_data = (torch_data(X_test), torch_data(y_test))
# Train on set 
model = PGBM()  
model.train(train_data, objective=mseloss_objective, metric=rmseloss_metric, params=params)
#% Point and probabilistic predictions. By default, 100 probabilistic estimates are created
yhat_point = model.predict(test_data[0])
yhat_dist = model.predict_dist(test_data[0])
# Scoring
rmse = model.metric(yhat_point, test_data[1])
crps = model.crps_ensemble(yhat_dist, test_data[1]).mean()    
# Print final scores
print(f'RMSE PGBM: {rmse:.2f}')
print(f'CRPS PGBM: {crps:.2f}')

In [ ]:
#%% Plot all samples
plt.rcParams.update({'font.size': 12})
plt.plot(y_test, 'o', label='Actual')
plt.plot(yhat_point.cpu(), 'ko', label='Point prediction PGBM')
plt.plot(yhat_dist.cpu().max(dim=0).values, 'k--', label='Max bound PGBM')
plt.plot(yhat_dist.cpu().min(dim=0).values, 'k--', label='Min bound PGBM')
plt.legend()